# Run this notebook on the Demand forecast cluster

In [0]:
%pip install lightgbm

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)

In [0]:
%run /Users/ebizindavyi@delhaize.be/Data_preparation/Parse_xls_files

In [0]:
%run /Users/ebizindavyi@delhaize.be/Evaluation/Evaluation_functions

In [0]:
import pandas as pd
import numpy as np
import datetime as dt
import lightgbm as lgb
from reusable.featurestore.features_schoolholidays import school_holidays
from pyspark.sql import functions as f


## Evaluation Params

In [0]:
actual_cols = ['Dry Actuals', 'Dry Actuals']
fc_cols = ['Dry Fc', 'LGBM_FC']
error_suffixes = ['Dry_WOW', 'Dry_LGBM']
error_to_calc = ["Bias", "AbsError", "SqError","RelAbsError"]

## Load inbound data

In [0]:
inbound_2022_dir = '/dbfs/mnt/dataplatform/acc/DataScience/sandbox/ebiz/Inbound_FC/data/input/Ecom-Ops-status_2022.xlsx'

In [0]:
inbound_2022_df = create_master_df(inbound_2022_dir)


In [0]:
# calendar_2022 = pd.DataFrame({'Dates':pd.date_range(dt.datetime(2022,1,3), periods=363, freq='d')})
# inbound_2022_dry = calendar_2022.merge(inbound_2022_df[['Dates','Dry Actuals']], on='Dates',how='left').fillna(0).copy()

### Get holidays

In [0]:
flemish_holidays = (
  school_holidays()
  .filter(
    (f.col('is_dutch_region_holiday')==1)
    &
    (f.col('holiday_name').isNull())
    )
  .withColumn('holiday', f.lit('flemish_region_holiday'))
  .withColumn('ds', f.to_date(f.col('date')))
  .select('holiday','ds')
  .toPandas()
)

walloon_holidays =(
  school_holidays()
  .filter(
    (f.col('is_french_region_holiday')==1)
    &
    (f.col('holiday_name').isNull())
    )
  .withColumn('holiday', f.lit('walloon_region_holiday'))
  .withColumn('ds', f.to_date(f.col('date')))
  .select('holiday','ds')
  .toPandas()
)

holidays = pd.concat((flemish_holidays, walloon_holidays))


## LightGBM


##Parameters

In [0]:
#lgbm parameter
backtest_start_date = pd.to_datetime('2022-01-03')
backtest_end_date =  pd.to_datetime('2022-09-30')

#shift periods to create features(lags)
shift_period_1 = 7
shift_period_2 = 14
shift_period_3 = 21

# Initialize lgbm parameters 
params = {
    'objective': 'regression',  
    'metric': 'rmse',           
    'boosting_type': 'gbdt',    
    # 'max_depth':2,      
}
# variables 
feature_col = ['x_7days', 'x_14days', 'x_21days']
pred_period=6
end_horizon = 7
fc_freq=7
training_period= 90

### Backtest

In [0]:
backtest_results = pd.DataFrame()

inbound_2022_train = inbound_2022_df[['Dates', 'Dry Actuals']].copy() #original filtered dataset, no sundays&NANs

inbound_2022_train['x_7days'] = inbound_2022_df['Dry Actuals'].shift(shift_period_1)
inbound_2022_train['x_14days'] = inbound_2022_df['Dry Actuals'].shift(shift_period_2)
inbound_2022_train['x_21days'] = inbound_2022_df['Dry Actuals'].shift(shift_period_3)

# Iterate through each backtesting period
while backtest_start_date + dt.timedelta(days=pred_period) <= backtest_end_date:
  print(f"Backtesting on: {backtest_start_date}")
  
  start_train_date = backtest_start_date
  end_train_date = start_train_date + dt.timedelta(days=training_period)
  print(f"Backtesting training ending on: {end_train_date}") 

  inbound_2022_dry_lgbm_train = inbound_2022_train[(inbound_2022_train['Dates'] >= start_train_date) &(inbound_2022_train['Dates'] < end_train_date)].copy()
  print(f"print the shape:{inbound_2022_dry_lgbm_train.shape} ")
  # Convert Dates column to datetime
  inbound_2022_dry_lgbm_train['Dates'] = pd.to_datetime(inbound_2022_dry_lgbm_train['Dates'])
    
  inbound_2022_dry_lgbm_train = inbound_2022_dry_lgbm_train.dropna()
  features_values_train=inbound_2022_dry_lgbm_train.iloc[:, -3:]

  # Filter the dates with values in Dry Actuals 
  features_train = features_values_train.copy() 
  target = inbound_2022_dry_lgbm_train['Dry Actuals']
  
  # Create LightGBM dataset
  lgb_train = lgb.Dataset(features_train, label=target)
    
  # Model training
  m = lgb.train(params, lgb_train)

  # Prediction for the current period
  future_dates = pd.date_range(start=end_train_date+dt.timedelta(days=14), periods=pred_period, freq='D')

  future_features = pd.DataFrame({'ds': future_dates})
  future_features['ds'] = pd.to_datetime(future_features['ds'])

  future_features = inbound_2022_train[inbound_2022_train['Dates'].isin(future_dates)].iloc[:, -3:].values ### Changed the source df
  prediction_dates = inbound_2022_train[inbound_2022_train['Dates'].isin(future_dates)]['Dates'] ### New variable to extract valid dates from the future date range
  
  #forecast using lgbm model m

  forecast = m.predict(future_features, num_iteration=m.best_iteration, predict_disable_shape_check=True)

  lgbm_pred = pd.DataFrame({'ds': prediction_dates, 'yhat': forecast}) ### Use dates form df rather than the original date range
    
  backtest_results = pd.concat([backtest_results, lgbm_pred])
    
  # Increase training period after each iterations 
  backtest_start_date += dt.timedelta(days=end_horizon)
 
print("Backtesting completed.")


Backtesting on: 2022-01-03 00:00:00
Backtesting training ending on: 2022-04-03 00:00:00
print the shape:(75, 5) 
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020837 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 57
[LightGBM] [Info] Number of data points in the train set: 54, number of used features: 3
[LightGBM] [Info] Start training from score 65105.407407
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

### Reformat backtest

In [0]:
backtest_results['Dates']= backtest_results['ds']
backtest_results['LGBM_FC']= backtest_results['yhat']
backtest_df = backtest_results[['Dates','LGBM_FC']].reset_index(drop=True).copy()
backtest_df['Dates'] = pd.to_datetime(backtest_df['Dates'])

### Join the backtest to the inbound

In [0]:
###Remove the data quality issues where FC is exactly te same as actuals

backtest_df_merge = pd.merge(inbound_2022_df[inbound_2022_df['Dry Fc']!=inbound_2022_df['Dry Actuals']],backtest_df, on="Dates", how='inner')[['Dates','Dry Actuals','Dry Fc','LGBM_FC']]
backtest_df_merge

,Dates,Dry Actuals,Dry Fc,LGBM_FC
0,2022-04-19,87006.0,81040.622023,69991.422135
1,2022-04-20,49021.0,58967.273526,69991.422135
2,2022-04-21,44875.0,59909.705493,73985.349642
3,2022-04-22,49405.0,57751.017683,73043.511549
4,2022-04-25,102000.0,90650.857878,67913.406370
...,...,...,...,...
173,2022-12-26,90911.0,95488.000000,71372.764649
174,2022-12-27,63764.0,71459.270000,55637.664183
175,2022-12-28,47680.0,71916.320000,58781.888013
176,2022-12-29,39552.0,60346.346000,61011.582847


## Evaluate

### Create error columns

In [0]:
for error in  error_to_calc:
  for i in range(len(actual_cols)):
    calculate_errors(df_eval= backtest_df_merge, actuals_col=actual_cols[i], fc_col= fc_cols[i],error_to_calculate=error,error_colname_suffix=error_suffixes[i])
    

In [0]:
backtest_df_merge

,Dates,Dry Actuals,Dry Fc,LGBM_FC,Bias_Dry_WOW,Bias_Dry_LGBM,AbsError_Dry_WOW,AbsError_Dry_LGBM,SqError_Dry_WOW,SqError_Dry_LGBM,RelAbsError_Dry_WOW,Outside_range_Dry_WOW,RelAbsError_Dry_LGBM,Outside_range_Dry_LGBM
0,2022-04-19,87006.0,81040.622023,69991.422135,-5965.377977,-17014.577865,5965.377977,17014.577865,3.558573e+07,2.894959e+08,0.068563,False,0.195556,True
1,2022-04-20,49021.0,58967.273526,69991.422135,9946.273526,20970.422135,9946.273526,20970.422135,9.892836e+07,4.397586e+08,0.202898,True,0.427784,True
2,2022-04-21,44875.0,59909.705493,73985.349642,15034.705493,29110.349642,15034.705493,29110.349642,2.260424e+08,8.474125e+08,0.335035,True,0.648699,True
3,2022-04-22,49405.0,57751.017683,73043.511549,8346.017683,23638.511549,8346.017683,23638.511549,6.965601e+07,5.587792e+08,0.168931,True,0.478464,True
4,2022-04-25,102000.0,90650.857878,67913.406370,-11349.142122,-34086.593630,11349.142122,34086.593630,1.288030e+08,1.161896e+09,0.111266,True,0.334182,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,2022-12-26,90911.0,95488.000000,71372.764649,4577.000000,-19538.235351,4577.000000,19538.235351,2.094893e+07,3.817426e+08,0.050346,False,0.214916,True
174,2022-12-27,63764.0,71459.270000,55637.664183,7695.270000,-8126.335817,7695.270000,8126.335817,5.921718e+07,6.603733e+07,0.120684,True,0.127444,True
175,2022-12-28,47680.0,71916.320000,58781.888013,24236.320000,11101.888013,24236.320000,11101.888013,5.873992e+08,1.232519e+08,0.508312,True,0.232842,True
176,2022-12-29,39552.0,60346.346000,61011.582847,20794.346000,21459.582847,20794.346000,21459.582847,4.324048e+08,4.605137e+08,0.525747,True,0.542566,True


In [0]:

days_out_range= backtest_df_merge.groupby('Dates').agg({'Outside_range_Dry_LGBM':'sum'}).reset_index()
days_out_range

,Dates,Outside_range_Dry_LGBM
0,2022-04-19,1
1,2022-04-20,1
2,2022-04-21,1
3,2022-04-22,1
4,2022-04-25,1
...,...,...
173,2022-12-26,1
174,2022-12-27,1
175,2022-12-28,1
176,2022-12-29,1


In [0]:
filter_df = days_out_range[days_out_range['Outside_range_Dry_LGBM'] != 0]
filter_df

,Dates,Outside_range_Dry_LGBM
0,2022-04-19,1
1,2022-04-20,1
2,2022-04-21,1
3,2022-04-22,1
4,2022-04-25,1
...,...,...
173,2022-12-26,1
174,2022-12-27,1
175,2022-12-28,1
176,2022-12-29,1


### Calculate global metrics

In [0]:
global_metrics_dry = describe_inbound_fc(inbound_fc=backtest_df_merge, cols_to_keep=['Dry Actuals','Dry Fc','LGBM_FC','Bias_Dry_WOW','Bias_Dry_LGBM', 'AbsError_Dry_WOW','AbsError_Dry_LGBM','SqError_Dry_WOW','SqError_Dry_LGBM','RelAbsError_Dry_WOW','RelAbsError_Dry_LGBM', 'Outside_range_Dry_WOW', 'Outside_range_Dry_LGBM'])

<command-203330119889549>:25: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



In [0]:
global_metrics_dry

,Metrics,Dry Actuals,Dry Fc,LGBM_FC,Bias_Dry_WOW,Bias_Dry_LGBM,AbsError_Dry_WOW,AbsError_Dry_LGBM,SqError_Dry_WOW,SqError_Dry_LGBM,RelAbsError_Dry_WOW,RelAbsError_Dry_LGBM,Outside_range_Dry_WOW,Outside_range_Dry_LGBM
0,count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,1.780000e+02,1.780000e+02,178.000000,178.000000,178,178
1,unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2
2,top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
3,freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107,133
4,first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,mean,67033.084270,68694.963125,62431.827492,1661.878855,-4601.256778,6541.775206,17295.883790,9.851329e+07,6.222027e+08,0.117187,0.257027,NaN,NaN
7,std,24859.765002,22163.902910,9776.509380,9812.869893,24585.093499,7485.536652,18024.436305,3.450056e+08,1.555479e+09,0.209084,0.290015,NaN,NaN
8,min,21742.000000,35712.572000,42376.879337,-59204.840000,-120540.461284,4.000000,262.737707,1.600000e+01,6.903110e+04,0.000081,0.003971,NaN,NaN
9,25%,50036.500000,55084.871122,54761.684444,-2567.530000,-14017.450481,2083.976504,5718.954696,4.345082e+06,3.270652e+07,0.030046,0.095695,NaN,NaN


### Visualize profiles

In [0]:
plot_two_inbound_fc(inbound_df=backtest_df_merge, actuals_col='Dry Actuals', fc_col_1='Dry Fc', fc_col_2= 'LGBM_FC', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')

### Visualize errors

In [0]:
plot_inbound_two_errors(inbound_df=backtest_df_merge, error_col_1='Bias_Dry_WOW', error_col_2= 'Bias_Dry_LGBM', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')

In [0]:
plot_inbound_two_errors(inbound_df=backtest_df_merge, error_col_1='AbsError_Dry_WOW', error_col_2= 'AbsError_Dry_LGBM', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')